In [14]:
import gspread
import pandas as pd

gc = gspread.service_account(filename="keys.json")

spreadsheet_id = "1JTHSO1t5IWqDOQxbd24vs_YMghd24ENxf4kbthUcBo8"
spreadsheet = gc.open_by_key(spreadsheet_id)

location_sheet = spreadsheet.worksheet("Location") 
inbox_sheet = spreadsheet.worksheet("Inbox")        
output_sheet = spreadsheet.worksheet("Output")    

location_data = location_sheet.get_all_records()
inbox_data = inbox_sheet.get_all_records()
output_data = output_sheet.get_all_records()

location_df = pd.DataFrame(location_data)
inbox_df = pd.DataFrame(inbox_data)
output_df = pd.DataFrame(output_data)


if not {"Location", "Booking Number"}.issubset(location_df.columns):
    raise ValueError("Location sheet must contain 'Location' and 'Booking Number' columns.")
if not {"Location", "Contact"}.issubset(inbox_df.columns):
    raise ValueError("Inbox sheet must contain 'Location' and 'Contact' columns.")

    

merged_df = pd.merge(inbox_df, location_df, on="Location", how="inner")


if not output_df.empty:
    
    merged_df = pd.concat([output_df, merged_df], ignore_index=True)
    
    merged_df = merged_df.drop_duplicates()


output_sheet.clear()  
output_sheet.update([merged_df.columns.values.tolist()] + merged_df.values.tolist())


print("Output sheet updated successfully!")

Output sheet updated successfully!
